In [1]:
import pandas as pd
import numpy as np
column_names = ["Date", "Price", "Open", "High", "Low"]

# Load data

In [2]:
path= 'TSLA.csv'
with open(path)as file:
    data=pd.read_csv(file,header=0,encoding='UTF-8')
    print("Data Loaded!")
    npdata=data.to_numpy()

Data Loaded!


# TASK1 查找NaN的位置返回并删除NaN所在行

##### 查找位置

In [3]:
# 从内到外依次为：
# 获取除了Data的列——>查找NaN所在的位置——>保留行号信息——>去重——>保存在position中
position = np.unique(np.where(np.isnan(npdata[:,1:].astype(float)))[0])

# 去除了标题行，并且标号从0开始，故转化为原来csv表需要加上2
print("NaN所在行数为")
print(position+2)

NaN所在行数为
[  6   7  13  14  20  21  27  28  34  35  41  42  48  49  55  56  62  63
  69  70  76  77  83  84  90  91  97  98 104 105 111 112 118 119 125 126
 132 133 139 140 146 147 153 154 160 161 167 168 174 175]


#### 删除NaN所在行


In [4]:
del_nan = data.drop(axis=0,index=position)
del_nan

Date    Price     Open     High      Low
0     2022/4/14   985.00   999.29  1012.71   982.19
1     2022/4/13  1022.37   981.08  1026.24   973.10
2     2022/4/12   986.95   997.64  1021.19   976.60
3     2022/4/11   975.93   980.40  1008.47   974.64
6      2022/4/8  1025.49  1043.21  1048.44  1022.43
...         ...      ...      ...      ...      ...
2531  2012/4/18     6.53     6.42     6.55     6.31
2532  2012/4/18     6.53     6.42     6.55     6.31
2533  2012/4/18     6.53     6.42     6.55     6.31
2534  2012/4/18     6.53     6.42     6.55     6.31
2535  2012/4/17     6.45     6.49     6.61     6.41

[2486 rows x 5 columns]

# TASK2 删除重复的日期数据并打印出删除的行数

In [5]:
# 将npdata用上一问去掉NaN的data frame更新
npdata = del_nan.to_numpy()

In [6]:
# 去重
# Args: unique_date是保留的日期，left_rows是保留的行
unique_date, left_rows = np.unique(npdata[:,0],return_index = True)

In [7]:
npdata

array([['2022/4/14', 985.0, 999.29, 1012.71, 982.19],
       ['2022/4/13', 1022.37, 981.08, 1026.24, 973.1],
       ['2022/4/12', 986.95, 997.64, 1021.19, 976.6],
       ...,
       ['2012/4/18', 6.53, 6.42, 6.55, 6.31],
       ['2012/4/18', 6.53, 6.42, 6.55, 6.31],
       ['2012/4/17', 6.45, 6.49, 6.61, 6.41]], dtype=object)

In [8]:
count = npdata[:,0].size-left_rows.size
print("删除了{}行".format(count))

删除了8行


In [9]:
# 将原来del_nan的行编号重新排列
del_nan = pd.DataFrame(npdata,columns = column_names)

In [10]:
# 获取删除的行数
del_rows = []
for i in range(npdata[:,0].size):
    if ((left_rows.__contains__(i))==False):
        del_rows = np.append(del_rows,i).astype(int)
print("有重复数据的行号为：",del_rows)
# 删除重复日期，保存在del_date中
del_date = del_nan.drop(del_rows)
print("删除重复日期成功！")

有重复数据的行号为： [2471 2472 2479 2480 2481 2482 2483 2484]
删除重复日期成功！


# TASK3 涨停跌停计算

In [11]:
column_names = ["Date", "Price", "Open", "High", "Low","Diff"]

In [12]:
npdata =del_date.to_numpy()

In [13]:
price = npdata[:,1].astype(float)
price_range = np.zeros(len(price))
for i in range(len(price_range)-1):
    price_range[i]=(price[i]-price[i+1])/price[i+1]
price_range = np.around(price_range,4)

In [14]:
# 涨停天数
trading_days = np.count_nonzero(price_range > 0.1)
trading_days
# 跌停天数
stopdown_days = np.count_nonzero(price_range < -0.1)
print("涨停天数为{}天，跌停天数为{}天".format(trading_days,stopdown_days))

涨停天数为38天，跌停天数为21天


In [15]:
diff = np.zeros(len(npdata))
for i in range(len(npdata)):
    diff[i] = npdata[i,3]-npdata[i,4]
np.around(diff,4)

array([30.52, 53.14, 44.59, ...,  0.19,  0.24,  0.2 ])

In [16]:
npdata=np.insert(npdata,5,diff,axis=1)

In [17]:
npdata

array([['2022/4/14', 985.0, 999.29, 1012.71, 982.19, 30.519999999999982],
       ['2022/4/13', 1022.37, 981.08, 1026.24, 973.1, 53.139999999999986],
       ['2022/4/12', 986.95, 997.64, 1021.19, 976.6, 44.59000000000003],
       ...,
       ['2012/4/19', 6.63, 6.55, 6.69, 6.5, 0.1900000000000004],
       ['2012/4/18', 6.53, 6.42, 6.55, 6.31, 0.2400000000000002],
       ['2012/4/17', 6.45, 6.49, 6.61, 6.41, 0.20000000000000018]],
      dtype=object)

In [18]:
save_csv = pd.DataFrame(npdata, columns=column_names)
save_csv
save_csv.to_csv('TSLA_processed.csv',index=False)